# Speakato sandbox - free model testing

In [13]:
import spacy
import pandas as pd
import numpy as np
import json
import keras.models

## Loading dataset and models

In [14]:
model_path = "../models/spkt-test"
dataset_path = "../examples/polish_commands_dataset"

with open(f"{model_path}/info.json", "r") as f:
    config = json.load(f)

with open(f"{dataset_path}/commands.txt", "r") as f:
    labels = [x.rstrip("\n") for x in f.readlines()]

config

{'language': 'pl',
 'spacy_model': 'pl_core_news_sm',
 'token_len': 96,
 'train_evaluation': [0.062287718057632446, 0.9811320900917053],
 'test_evaluation': [0.07174625992774963, 0.95652174949646]}

In [15]:
nlp = spacy.load(config["spacy_model"])
data = pd.read_json(f"{dataset_path}/dataset.json")
data

,text,command
0,Dzień dobry,greeting
1,Dobry wieczór,greeting
2,Cześć,greeting
3,Hej,greeting
4,Siema,greeting
...,...,...
71,Stwórz nowy folder,create
72,Wejdź do folderu,open
73,Otwórz folder,open
74,Wróć do folderu,open


In [16]:
model = keras.models.load_model(f"{model_path}/model")

## Evaluation and comparison of the results

In [17]:
X_raw = data["text"]
X = np.array([nlp(x).vector for x in X_raw])
y = pd.get_dummies(data[["command"]]).values

model.evaluate(X, y)

3/3 [==============================] - 0s 2ms/step - loss: 1.7899 - accuracy: 0.4868


[1.7899340391159058, 0.4868420958518982]

In [10]:
y_predicted = model(X)

In [11]:
for i, x in enumerate(X_raw):
    correct_index = y[i].tolist().index(max(y[i]))
    recognized_index = y_predicted[i].numpy().tolist().index(max(y_predicted[i]))
    print(f"{x}\nRecognized: {recognized_index}\nCorrect: {correct_index}\n")

Dzień dobry
Recognized: 4
Correct: 4

Dobry wieczór
Recognized: 4
Correct: 4

Cześć
Recognized: 4
Correct: 4

Hej
Recognized: 4
Correct: 4

Siema
Recognized: 4
Correct: 4

Elo, co tam
Recognized: 4
Correct: 4

Witaj
Recognized: 5
Correct: 4

Witam
Recognized: 4
Correct: 4

Czołem
Recognized: 4
Correct: 4

Joł
Recognized: 4
Correct: 4

Jak się masz
Recognized: 4
Correct: 4

Otwórz aplikację spotify
Recognized: 5
Correct: 5

Uruchom grę
Recognized: 1
Correct: 5

Otwórz menadżera zadań
Recognized: 5
Correct: 5

Uruchom wiersz poleceń
Recognized: 9
Correct: 5

Włącz chrome
Recognized: 4
Correct: 5

Otwórz notatnik
Recognized: 5
Correct: 5

Włącz mi aplikacje notatnik
Recognized: 3
Correct: 5

Uruchom przeglądarkę
Recognized: 5
Correct: 5

Uruchom kalkulator
Recognized: 5
Correct: 5

Odpal kalkulator
Recognized: 5
Correct: 5

Uruchom aplikacje
Recognized: 9
Correct: 5

Otwórz panel sterowania
Recognized: 5
Correct: 5

Otwórz ustawienia systemu
Recognized: 2
Correct: 5

Otwórz galerię zdjęć


In [21]:
Predict = np.array([nlp(x).vector for x in ["Hej"]])
y_predicted = model(Predict)
recognized_index = y_predicted[0].numpy().tolist().index(max(y_predicted[0]))
print(recognized_index)

4
